In [1]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv("c:/Users/tquan/Downloads/ner_dataset.csv", encoding= 'latin1')
data = data.fillna(method='ffill')
data

C:\Users\tquan\AppData\Local\Temp\ipykernel_15876\213452595.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method='ffill')


,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O


In [7]:
agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                    s["POS"].values.tolist(),
                                                    s["Tag"].values.tolist())]

grouped = data.groupby("Sentence #").apply(agg_func)
sentences = [s for s in grouped]
sentence = [[w[0] for w in s] for s in sentences]